In [1]:
import torch
import torch.nn as nn

from models import TextEncoder, ImageEncoder
from datasets import Recipe1MDataset
from torch.utils.data import DataLoader
from transformers import BertTokenizer

In [2]:
temp = torch.load('saved_models/model.pt', map_location='cpu')
img_encoder = ImageEncoder()
txt_encoder = TextEncoder(2, 2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
img_encoder.load_state_dict(temp['img_encoder'])
txt_encoder.load_state_dict(temp['txt_encoder'])

<All keys matched successfully>

In [4]:
data = Recipe1MDataset(part='train')

In [5]:
data[0]

('Kombu Tea Grilled Chicken Thigh\nchicken thighs\ntea\nwhite pepper\nPierce the skin of the chicken with a fork or knife.\nSprinkle with kombu tea evenly on both sides of the chicken, about 1 teaspoon per chicken thigh.\nBrown the skin side of the chicken first over high heat until golden brown.\nSprinkle some pepper on the meat just before flipping over.\nThen brown the other side until golden brown.',
 tensor([[[ 0.6627,  0.6706,  0.6784,  ...,  0.7961,  0.7961,  0.7647],
          [ 0.6627,  0.6706,  0.6706,  ...,  0.7804,  0.7882,  0.8039],
          [ 0.6863,  0.6863,  0.6784,  ...,  0.7961,  0.7882,  0.8118],
          ...,
          [ 0.1137,  0.0902,  0.0824,  ...,  0.0902,  0.2706,  0.8824],
          [ 0.1373,  0.1059,  0.1059,  ...,  0.1216,  0.0275,  0.6314],
          [ 0.1686,  0.1216,  0.1059,  ...,  0.1686,  0.0118,  0.1765]],
 
         [[ 0.5529,  0.5608,  0.5686,  ...,  0.6863,  0.6863,  0.6549],
          [ 0.5529,  0.5608,  0.5608,  ...,  0.6706,  0.6784,  0.6941]

In [6]:
dataloader = DataLoader(data, batch_size=8, shuffle=False)

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [8]:
import pickle


def pickler(obj, fpath, mode):
    with open(fpath, mode) as f:
        pickle.dump(obj, f)

In [9]:
from tqdm import tqdm


device = 'cuda:7'
img_encoder = img_encoder.to(device)
img_encoder.eval()
with torch.no_grad():
    for txt, image in tqdm(dataloader):
        txt_input = tokenizer(txt, truncation=True, padding=True, return_tensors="pt")
        # txt_cls_output, other_tokens = txt_encoder(**txt_input)
        img_patches = img_encoder(image.to(device))
        pickler(img_patches.cpu().numpy(), 'saved_models/image_embeddings.pkl', 'ab')

  0%|                                                   | 12/56541 [00:09<12:10:43,  1.29it/s]


KeyboardInterrupt: 